# PyCity Schools Analysis


In [218]:
# Import dependencies
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [219]:
# Total Number of schools
total_schools = len(df.school_name.unique())
total_schools

15

In [220]:
# Calculate the total number of students
student_count = len(df.student_name)
student_count

39170

In [221]:
# Calculate the total budget

df_school = df.drop_duplicates("School ID")
total_budget = sum(df_school.budget)

total_budget

24649428

In [222]:
# Calculate the average (mean) math score
average_math_score = df.math_score.mean()
average_math_score

78.98537145774827

In [223]:
# Calculate the average (mean) reading score
average_reading_score = df.reading_score.mean()
average_reading_score

81.87784018381414

In [224]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = df[(df["math_score"] >= 70)].count()["Student ID"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [225]:
# Calculate the percentage of students who passed reading
passing_reading_count = df[(df["reading_score"] >= 70)].count()["Student ID"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [226]:
# Calculate the percentage of students that passed math and reading
passing_math_reading_count = df[
    (df["math_score"] >= 70) & (df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [227]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": total_schools,
    "Total Students": student_count,
    "Total Budget": total_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": passing_math_percentage,
    "% Passing Reading": passing_reading_percentage,
    "% Overall Passing": overall_passing_rate
}, index=[0])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [ ]:
## School Summary

In [228]:
#School names
school_name = df.set_index('school_name', drop=True)
# School types
school_types = school_data.set_index('school_name')['type']

In [229]:
# Total student count per school
school_count = df.groupby(['school_name'])['Student ID'].count()


In [230]:
# Total school budget
total_school_budget = pd.DataFrame(school_data.set_index('school_name')['budget'])
per_school_capita = pd.DataFrame(total_school_budget.budget / school_count, columns=['Per Student Budget'])

In [231]:
# Calculate the average test scores per school
per_school_math = school_name.groupby(by='school_name')['math_score'].mean()
per_school_reading = school_name.groupby(by='school_name')['reading_score'].mean()

In [232]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_name[school_name.math_score >= 70]
school_students_passing_math = students_passing_math.groupby(by='school_name').size()

In [233]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_name[school_name.reading_score >= 70]
school_students_passing_reading = students_passing_reading.groupby(by='school_name').size()

In [234]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = df[
    (df["reading_score"] >= 70) & (df["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [235]:
# Use the provided code to calculate the passing rates
per_school_passing_math = pd.DataFrame(school_students_passing_math / school_count * 100, 
                                       columns = ['% Passing Math'])
per_school_passing_reading = pd.DataFrame(school_students_passing_reading / school_count * 100,
                                          columns = ['% Passing Reading'])
overall_passing_rate = pd.DataFrame(school_students_passing_math_and_reading / school_count * 100, 
                                    columns = ['% Overall Passing'])

In [236]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.merge(school_types, school_count, on='school_name').merge(
    total_school_budget, on='school_name').merge(
    per_school_capita, on='school_name').merge(
    per_school_math, on='school_name').merge(
    per_school_reading, on='school_name').merge(
    per_school_passing_math, on='school_name').merge(
    per_school_passing_reading, on='school_name').merge(overall_passing_rate, on='school_name')

per_school_summary.sort_index(inplace=True)

#hide index name
per_school_summary.index.name = None

#Rename columns
per_school_summary.rename(columns={'type': 'School Type',
                                  'Student ID': 'Total Students',
                                  'budget': 'Total School Budget',
                                  'math_score': 'Average Math Score',
                                  'reading_score': 'Average Reading Score'}, inplace=True)
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
